In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Standardize result

In [ ]:
import json
import re
import os

def map_json_to_format(data):
    mapping = {
      # 0 - Primeira Habilitação
      "primeira_habilitação": 0,
      "primeira_habilitacao": 0,
      "primeira habilitação": 0,
      "pela_primeira_habilitação": 0,
      "1ª_habilitação": 0,
      "data_primeira_habilitação": 0,

      # 1 - Categoria
      "categoria": 1,
      "categoría": 1,

      # 2 - CPF
      "cpf": 2,
      "CPF": 2,

      # 3 - Data de Nascimento
      "data_de_nascimento": 3,
      "data_nascimento": 3,
      "data de nascimento": 3,
      "data_nascimento_2": 3,
      "data_de_nascimento_2": 3,
      "data_de_nascimento_duplicada": 3,
      "data_de_nascimento_repetida": 3,
      "data_nascimento_repete": 3,
      "data_nascimento_repetida": 3,
      "data_nascimento_rep": 3,
      "data_nascimento_secundária": 3,
      "data_nascimento_acc": 3,
      "data_nascimento_registro": 3,
      "data de nascimento (filiação)": 3,

      # 4 - Documento de Identidade/Órgão Emissor/UF
      "documento_de_identidade": 4,
      "documento_identidade": 4,
      "documento de identidade": 4,
      "doc_identidade": 4,
      "doc. identidade": 4,
      "órgão_emissor": 4,
      "orgao_emissor": 4,
      "organização_emissor": 4,
      "órgão emissor": 4,
      "orgão_emissor": 4,
      "orgão emissor": 4,
      "órgao_emissor": 4,
      "org. emissor": 4,
      "UF": 4,
      "uf": 4,

      # 5 - Filiação
      "filiação": 5,
      "filiacao": 5,
      "afiliação": 5,
      "filacao": 5,

      # 6 - Número do Registro
      "número_do_registro": 6,
      "número_registro": 6,
      "n_registro": 6,
      "número do registro": 6,
      "numero_do_registro": 6,
      "numero_registro": 6,

      # 7 - Nome
      "nome": 7,

      # 8 - Validade
      "validade": 8,

      # 9 - Número Válido
      "número_válido": 9,
      "número_valido": 9,
      "numero_valido": 9,
      "numero_valido_lateral": 9,
      "número_válido_lateral": 9,
      "numero_valido_vertical": 9,
      "número_válido_vertical": 9,

      # 10 - Permissão
      "permissão": 10,
      "permissao": 10,

      # 11 - ACC
      "ACC": 11,
      "acc": 11,
      "accent": 11,

      # Outras datas
      "data_cadastro": 12,
      "data_de_biometria": 12,
      "data_de_registro": 12,
      "data_documento": 12
  }


    result_dict = {}

    for key, value in mapping.items():
        if key in data and data[key]:
            if isinstance(data[key], list):
                if all(isinstance(item, dict) for item in data[key]):
                    data[key] = ' '.join(
                        re.sub(r'\\', '', value) for obj in data[key] for value in obj.values()
                    )
                else:
                    data[key] = ' '.join(
                        item.replace('\\', '') for item in data[key]
                    )
            elif isinstance(data[key], dict):
                data[key] = ' '.join(
                    re.sub(r'\\', '', value) for value in data[key].values()
                )
            else:
                data[key] = data[key].replace('\\', '')
            print(data[key])
            if key in  {'filiação', 'filiacao', 'afiliação', 'filacao'} :
              if isinstance(data[key], list):
                if all(isinstance(item, dict) for item in data[key]):
                    data[key] = ' '.join(
                        re.sub(r'[\n,;\\]', '', value) for obj in data[key] for value in obj.values()
                    )
                else:
                    data[key] = ' '.join(
                        item.replace('\n', ' ').replace(',', '').replace(';', '').replace('\\', '')
                        for item in data[key]
                    )
              elif isinstance(data[key], dict):
                  data[key] = ' '.join(
                      re.sub(r'[\n,;\\]', '', value) for value in data[key].values()
                  )
              else:
                  data[key] = data[key].replace('\n', ' ').replace(',', '').replace(';', '').replace('\\', '').replace('PAI:', '').replace('MÃE:', '')
            if value in result_dict:
                result_dict[value] += f" {data[key]}"
            else:
                result_dict[value] = f'{data[key]}'

    result = [f'{key} "{value}"' for key, value in result_dict.items()]
    print(result)
    return result


def extract_json_from_txt(txt_content):
    json_match = re.search(r'\{.*\}', txt_content, re.DOTALL)

    if json_match:
        cleaned_json_str = json_match.group().replace('\\', '\\\\')
        return json.loads(cleaned_json_str)
    else:
        print("No JSON blocks found in file")

def process_and_save_txt_file(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        txt_content = file.read()

    data = extract_json_from_txt(txt_content)

    if (data):
      formatted_data = map_json_to_format(data)

      output_dir = os.path.dirname(output_file_path)
      if not os.path.exists(output_dir):
          os.makedirs(output_dir)

      with open(output_file_path, 'w', encoding='utf-8') as file:
          for line in formatted_data:
              file.write(line + '\n')

# Path to input (gpt return) and output (formatted return) folders
input_base_dir = '/content/drive/MyDrive/GPT/resultados_gpt_mini'
output_base_dir = '/content/drive/MyDrive/GPT/resultados_gpt_mini_formatado'

# Traverse all subfolders of the input directory
for folder_name in os.listdir(input_base_dir):
    input_folder_path = os.path.join(input_base_dir, folder_name)

    if os.path.isdir(input_folder_path):
        input_file_path = os.path.join(input_folder_path, 'hipotese.txt')

        if os.path.exists(input_file_path):
            output_folder_path = os.path.join(output_base_dir, folder_name)
            output_file_path = os.path.join(output_folder_path, 'resultado.txt')

            print(input_file_path)
            process_and_save_txt_file(input_file_path, output_file_path)

        else:
            print(f"File 'hipotese.txt' not found in: {input_folder_path}")


# Find attributes in results

In [ ]:
import json
import re
import os

# Function to extract the JSON from within the text file
def extract_json_from_txt(txt_content):
    json_match = re.search(r'\{.*\}', txt_content, re.DOTALL)
    if json_match:
        return json.loads(json_match.group())
    else:
        raise ValueError("Nenhum bloco JSON encontrado no arquivo")

# Function to loop through all txt files and identify attribute names in JSON
def find_unique_attributes_in_json(base_dir):
    unique_attributes = set()  # Use a set to ensure attributes are unique

    # Cycle through all subfolders and files in the base folder
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)

        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.txt'):
                    file_path = os.path.join(folder_path, file_name)

                    with open(file_path, 'r', encoding='utf-8') as file:
                        txt_content = file.read()

                    try:
                        # Extract JSON and get attributes (keys)
                        data = extract_json_from_txt(txt_content)
                        unique_attributes.update(data.keys())
                    except ValueError as e:
                        print(f"Error processing file {file_path}: {e}")

    return unique_attributes

input_base_dir = '/content/drive/MyDrive/resultados_gpt_mini'

# Identify all unique attributes
unique_attributes = find_unique_attributes_in_json(input_base_dir)

# Display the names of unique attributes found
print("Unique attributes found in JSON:")
for attribute in sorted(unique_attributes):
    print(attribute)
